# Smart signatures with ASA
#### 06.3 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Use Smart Signatures with ASAs
* Design a contract for token burning

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [ ]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

#### Check Purestake API

In [ ]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

In [ ]:
asset_holdings_df(algod_client,MyAlgo['public'])

## Burning
Burning seems simple: send money to a contract that always says "no".<br>
**Question:** how can a contract that always says "no" opt into an ASA?

#### Step 1: Define conditions in Pyteal
* Want to design a Burn contract for WSC coin
* Need WSC coin index
* For all Txn fields, see https://pyteal.readthedocs.io/en/stable/accessing_transaction_field.html

In [ ]:
# prepare the burn condition
WSC_idx =  71140107                     # <---------- change this to your WSC coin !!!!!

burn_cond = And (
    Txn.type_enum() == TxnType.AssetTransfer,        # Must be an "asset transfer" transaction
    Txn.amount() == Int(0),                          # Do not pay out ALGOs
    Txn.asset_amount() == Int(0),                    # Do also not pay out ASAs
    Txn.xfer_asset() == Int(WSC_idx)                 # Specific asset index
)

# prepare random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = Txn.fee() <= Int(1000)

safety_cond = And(
    # Txn.type_enum() == TxnType.Payment,             # This standard payment condition makes no senese here
    Txn.close_remainder_to() == Global.zero_address(),
    Txn.rekey_to() == Global.zero_address(),
    )
    
burn_pyteal = And(
    burn_cond,
    random_cond, 
    fee_cond, 
    safety_cond
    )

##### Step 2: Compile PyTeal -> Teal

In [ ]:
burn_teal = compileTeal(burn_pyteal, Mode.Signature, version=3)
print(burn_teal)

#### Step 3: Compile Teal -> Bytecode for AVM

In [ ]:
Burn = algod_client.compile(burn_teal)

#### Step 4: Fund burning contract
* The Burn contract has to pay TX fees for the opt-in transaction

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(0.1*1e6) + int(0.1*1e6) + int(0.001*1e6)   # Min holdings + min holdings for 1 ASA + TX fee
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Burn['hash'], amt=amt)

# Step 2+3+4: sign and send and wait ...
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 5: Burn contract opts-into WSC coin to allow burning
* This is an AssetTransferTx, that is signed by the Smart Signature
* Remember, opt-in is a transfer of zero units of an ASA to oneself

In [ ]:
# Step 5.1: Prepare
sp = algod_client.suggested_params()
txn = AssetTransferTxn(Burn['hash'], sp, Burn['hash'], 0, WSC_idx)

# Step 5.2: Sign
encodedProg = Burn['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 5.3 Send
txid = algod_client.send_transaction(stxn)

# Step 5.4 Wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

print('http://testnet.algoexplorer.io/tx/'+txid)

## The Burn contract is now ready for use

#### Manual burn transaction
* MyAlgo burns 8 WSC coins
* Simple AssetTransferTxn

In [ ]:
# WAIT a minute ... how many decimals does the WSC have?
WSC_decimals = algod_client.asset_info(WSC_idx)['params']['decimals']
print(WSC_decimals)

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int( 8 * 10**WSC_decimals )                   # <---------8 WSC coins in SMALL unit

txn = AssetTransferTxn(sender=MyAlgo['public'], sp=sp, 
                       receiver=Burn['hash'], amt=amt,
                       index=WSC_idx)

# Step 2+3+4: sign and send and wait ...
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

### QR code for burning
* Burning via QR code

In [ ]:
# URL for burning WITH asset index
url = 'algorand://' + Burn['hash'] + '?amount=1000&asset='+str(WSC_idx)+'&note=Burning'
print(url)

In [ ]:
import qrcode
qr = qrcode.QRCode(version=1,box_size=5,border=4)
qr.add_data(url)
qr.make(fit=True)
qr.make_image(fill_color="black", back_color="white")

## Exercise 1
* Reconsider the **burn condition** and discuss possible safety issues. 
* Espeically, discuss the fact that repeated transactions with zero ALGOs or with zero WSC are possible

```python
burn_cond = And (
    Txn.type_enum() == TxnType.AssetTransfer,        # Must be an "asset transfer" TX
    Txn.amount() == Int(0),                          # Do not pay out ALGOs
    Txn.asset_amount() == Int(0),                    # Do also not pay out ASAs
    Txn.xfer_asset() == Int(WSC_idx)                 # Specific asset index
)

```

## Exercise 2
* Reconsider the **safety condition** 
* Why was one common safety condition commented out?

```python
safety_cond = And(
    # Txn.type_enum() == TxnType.Payment,                     # <--- why???
    Txn.close_remainder_to() == Global.zero_address(),
    Txn.rekey_to() == Global.zero_address(),
    )
```